In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
#import matplotlib.pyplot as plt
import time
import os
import copy
from torchvision.datasets import ImageFolder
from torch.utils.data.sampler import SubsetRandomSampler
from train_snippets import train_model
import ipdb
#from visualize_images import imshow

In [4]:
import logging
logging.basicConfig(filename='train.log', level=logging.INFO, filemode='w', format='%(asctime)s %(levelname)-8s %(message)s',
                              datefmt='%Y-%m-%d %H:%M:%S')

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}
mask_tensor = mask_tensor = torch.Tensor([
        [1,0,0],
        [1,0,1],
        [0,1,1],
        [0,1,0],
        [0,0,1]
    ])



In [4]:
dataset = ImageFolder('/home/hthieu/AICityChallenge2019/data/Track2Data/vehicle_views_v3/', transform=data_transforms['train'],
                     target_transform=lambda x: mask_tensor[x])
print(dataset)

Dataset ImageFolder
    Number of datapoints: 15343
    Root Location: /home/hthieu/AICityChallenge2019/data/Track2Data/vehicle_views_v3/
    Transforms (if any): Compose(
                             RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                             RandomHorizontalFlip(p=0.5)
                             ToTensor()
                             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                         )
    Target Transforms (if any): <function <lambda> at 0x7ff71a9a2840>


In [5]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size],)
val_dataset.dataset = copy.copy(dataset) #copy, so it can have other transformation.
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']

dataloaders = {}
dataloaders['train'] = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
dataloaders['val'] = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=8)

In [6]:
logging.info(dataloaders['train'].dataset)
logging.info('*'*20)
logging.info(dataloaders['val'].dataset)
logging.info('logged')

In [6]:
N_CLASSES = 3

model_ft = models.resnet152(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, N_CLASSES)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

criterion = nn.BCEWithLogitsLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters())

# Decay LR by a factor of 0.1 every 10 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft,mode='min',factor=0.1,patience=10,threshold=0.0001)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, lr_scheduler,dataloaders=dataloaders,
                       num_epochs=100, device=device)

Epoch 0/99
----------
train Loss: 0.3931 Acc: 0.8368
val Loss: 0.2361 Acc: 0.9260
saving checkpoint as best model

Epoch 1/99
----------
train Loss: 0.2929 Acc: 0.8816
val Loss: 0.1992 Acc: 0.9333
saving checkpoint as best model

Epoch 2/99
----------
train Loss: 0.2670 Acc: 0.8908
val Loss: 0.1975 Acc: 0.9298

Epoch 3/99
----------
train Loss: 0.2569 Acc: 0.8953
val Loss: 0.1700 Acc: 0.9400
saving checkpoint as best model

Epoch 4/99
----------
train Loss: 0.2518 Acc: 0.8985
val Loss: 0.1707 Acc: 0.9381

Epoch 5/99
----------
train Loss: 0.2456 Acc: 0.9007
val Loss: 0.1547 Acc: 0.9466
saving checkpoint as best model

Epoch 6/99
----------
train Loss: 0.2456 Acc: 0.8996
val Loss: 0.1572 Acc: 0.9446

Epoch 7/99
----------
train Loss: 0.2368 Acc: 0.9044
val Loss: 0.1655 Acc: 0.9415

Epoch 8/99
----------
train Loss: 0.2337 Acc: 0.9049
val Loss: 0.1473 Acc: 0.9476
saving checkpoint as best model

Epoch 9/99
----------
train Loss: 0.2327 Acc: 0.9065
val Loss: 0.1563 Acc: 0.9435

Epoch 10/9

# Test

In [7]:
best = torch.load('./model_best.pth.tar')
model_ft.load_state_dict(best['state_dict'])

In [8]:
print(best['best_acc'])
print(best['epoch'])

tensor(0.9646, device='cuda:0')
83


In [11]:
print(model_ft)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [14]:
model = model_ft
model.eval()   # Set model to evaluate mode
running_loss = 0.0
running_corrects = 0
dataset_sizes = {x: len(dataloaders[x].dataset) for x in ['train', 'val']}
predicts = []

running_corrects = []
i=0
for inputs, labels in dataloaders['val']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    i+=1
    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        outputs = torch.sigmoid(outputs)
        preds = (outputs >= 0.5).float()
        running_corrects.append(torch.sum(preds == labels.data, dim=0))
    # statistics


running_corrects = torch.stack(running_corrects,dim=0)
epoch_acc = torch.mean(torch.sum(running_corrects,dim=0).float() / float(dataset_sizes['val'])) #mean acc

In [74]:
inputs, _ = val_dataset[6000]
classes = torch.sigmoid(model_ft(torch.unsqueeze(inputs,dim=0).cuda()))>=0.5
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[str(x) for x in classes])

IndexError: index 6000 is out of bounds for dimension 0 with size 3069

In [62]:
torch.unsqueeze(inputs,dim=0).shape

torch.Size([1, 3, 224, 224])

In [93]:
dts_test = ImageFolder('/home/bnminh/projects/ai2/datasets/ai_dataset/DATA/image_test_root/',transform=data_transforms['val'],
                     target_transform=lambda x: mask_tensor[x])
dtl_test = DataLoader(dts_test,batch_size=64, shuffle=False)

In [116]:
outputs_accumulated = []
for inputs, labels in dtl_test:
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        outputs = torch.sigmoid(outputs)
        outputs_accumulated.append(outputs)
    # statistics

In [117]:
outputs_accumulated = torch.cat(outputs_accumulated,dim=0)

In [118]:
outputs_accumulated.shape

torch.Size([18290, 3])

In [120]:
outputs_accumulated_np = outputs_accumulated.cpu().numpy()

In [122]:
np.savetxt('test_view_score.txt',outputs_accumulated_np,fmt="%.4f")

# test best imgs

In [163]:
import pandas as pd
from PIL import Image
class CSVDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.csv = np.genfromtxt(csv_file, delimiter=',', dtype='str')

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.csv[idx, 1])
        image = Image.open(img_name)
        
       
        if self.transform:
            sample = self.transform(image)

        return sample

In [164]:
test_best_dts = CSVDataset(csv_file='/home/hthieu/AICityChallenge2019/triplet-reid/data/track2_test_best_imgs.csv',
                          root_dir='/home/bnminh/projects/ai2/datasets/ai_dataset/DATA/image_test_root/image_test',
                          transform=data_transforms['val'])

In [169]:

dtl_test_best = DataLoader(test_best_dts,batch_size=64, shuffle=False)

In [170]:
outputs_accumulated = []
for inputs in dtl_test_best:
    inputs = inputs.to(device)
    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        outputs = torch.sigmoid(outputs)
        outputs_accumulated.append(outputs)
    # statistics

In [172]:
outputs_accumulated = torch.cat(outputs_accumulated,dim=0)
outputs_accumulated_np = outputs_accumulated.cpu().numpy()

In [174]:
np.savetxt('test_best_view_score.txt',outputs_accumulated_np,fmt="%.4f")